## Erzincan Elevation, Türkiye (2019)

This area elevation map is processed in the ESA tutorial available at: http://step.esa.int/docs/tutorials/S1TBX%20DEM%20generation%20with%20Sentinel-1%20IW%20Tutorial.pdf

The InSAR.dev ecosystem, the PyGMTSAR InSAR library, the Geomed3D geophysical inversion library, and N-Cube 3D/4D GIS data visualization (among others) are open-source projects I develop in my free time.

I hold a Master’s degree in STEM, specializing in radio physics. In 2004, I received first prize in the All-Russian Physics Competition for significant results in forward and inverse modeling for nonlinear optics and holography. These skills are also applicable to modeling gravity, magnetic, and thermal fields, as well as satellite interferometry processing.

With 20 years of experience as a data scientist and software developer, I have contributed to scientific and industrial development through government contracts, university projects, and work with companies including LG Corp and Google Inc.

You can support my work on Patreon, where I share updates on my projects, publications, use cases, examples, and other useful information. For research and development services and support, please visit my profile on the freelance platform Upwork.

You can support my work on [Patreon](https://www.patreon.com/pechnikov), where I share updates on my projects, publications, use cases, examples, and other useful information. For research and development services and support, please visit my profile on the freelance platform [Upwork](https://www.upwork.com) or reach out to me directly.

### Resources
- Google Colab Pro notebooks and articles on [Patreon](https://www.patreon.com/pechnikov),
- Google Colab notebooks on [GitHub](https://github.com),
- Docker Images on [DockerHub](https://hub.docker.com),
- Geological Models on [YouTube](https://www.youtube.com),
- VR/AR Geological Models on [GitHub](https://github.com),
- Live updates and announcements on [LinkedIn](https://www.linkedin.com/in/alexey-pechnikov/).

© Alexey Pechnikov, 2025

$\large\color{blue}{\text{Hint: Use menu Cell} \to \text{Run All or Runtime} \to \text{Complete All or Runtime} \to \text{Run All}}$
$\large\color{blue}{\text{(depending of your localization settings) to execute the entire notebook}}$

# Stage 1. InSAR.dev-PyGMTSAR: A Python package for InSAR pre-processing with PyGMTSAR

Convert Sentinel-1 SLC data using GMTSAR binaries into a geocoded, cloud-ready Zarr dataset on Google Colab or in a Docker container. The output can be hosted on GitHub or any file storage as a set of files or a single ZIP archive.

For Stage 2 InSAR analysis, only a pure-Python package is required—no binary installation needed—so processing can run on any Windows, macOS, or Linux host.

## Google Colab Installation

Install InSAR.dev Python libraries and required GMTSAR binaries

In [ ]:
import sys
# install the exact commit (no pip cache)
!{sys.executable} -m pip install --no-cache-dir \
  "git+https://github.com/AlexeyPechnikov/InSARdev.git#subdirectory=insardev_toolkit"
!{sys.executable} -m pip install --no-cache-dir \
  "git+https://github.com/AlexeyPechnikov/InSARdev.git#subdirectory=insardev_pygmtsar"
!{sys.executable} -m pip install --no-cache-dir \
  "git+https://github.com/AlexeyPechnikov/InSARdev.git#subdirectory=insardev"

In [ ]:
import platform, sys, os
if 'google.colab' in sys.modules:
    # script URL: https://github.com/AlexeyPechnikov/InSARdev/blob/main/insardev_pygmtsar/insardev_pygmtsar/data/google_colab.sh
    import importlib.resources as resources
    with resources.as_file(resources.files('insardev_pygmtsar.data') / 'google_colab.sh') as google_colab_script_filename:
        !sh {google_colab_script_filename}
    from google.colab import output
    output.enable_custom_widget_manager()

# specify GMTSAR installation path
PATH = os.environ['PATH']
if PATH.find('GMTSAR') == -1:
    PATH = os.environ['PATH'] + ':/usr/local/GMTSAR/bin/'
    %env PATH {PATH}

## Load and Setup Python Modules

In [ ]:
import geopandas as gpd
import json
import matplotlib.pyplot as plt
%matplotlib inline
# setup dark theme
from insardev.UI import UI
UI('dark')

In [ ]:
# print versions
from insardev_pygmtsar import __version__ as pygmtsar_version
from insardev_toolkit import __version__ as toolkit_version
print("insardev_pygmtsar version:", pygmtsar_version)
print("insardev_toolkit version:", toolkit_version)
# import modules to be used in the notebook
from insardev_pygmtsar import S1
from insardev_toolkit import EOF, ASF, Tiles, XYZTiles

## Specify Sentinel-1 SLC Bursts and Area

### Descending Orbit Configuration

https://search.asf.alaska.edu/#/?dataset=SENTINEL-1%20BURSTS&polygon=LINESTRING(39.5%2040,40%2039.5)&start=2019-07-01T17:00:00Z&end=2019-07-02T16:59:59Z&resultsLoaded=true&granule=S1_262885_IW2_20190702T032452_VV_69C5-BURST&zoom=8.867&center=39.720,39.246&polarizations=VV&flightDirs=Descending

https://search.asf.alaska.edu/#/?dataset=SENTINEL-1%20BURSTS&polygon=LINESTRING(39.5%2040,40%2039.5)&start=2019-07-07T17:00:00Z&end=2019-07-09T16:59:59Z&resultsLoaded=true&granule=S1_262886_IW2_20190708T032537_VV_33CA-BURST&zoom=8.867&center=39.720,39.246&polarizations=VV&flightDirs=Descending

In [ ]:
# Specify bursts to download
BURSTS = """
S1_262887_IW2_20190702T032458_VV_69C5-BURST
S1_262886_IW2_20190702T032455_VV_69C5-BURST
S1_262885_IW2_20190702T032452_VV_69C5-BURST

S1_262887_IW2_20190708T032540_VV_33CA-BURST
S1_262886_IW2_20190708T032537_VV_33CA-BURST
S1_262885_IW2_20190708T032534_VV_33CA-BURST
"""

In [ ]:
geojson = '''
{
  "type": "Feature",
  "geometry": {
    "type": "LineString",
    "coordinates": [[39.5, 40], [40, 39.5]]
  },
  "properties": {}
}
'''
POI = gpd.GeoDataFrame.from_features([json.loads(geojson)])

## Specify Directories

In [ ]:
# directory where Sentinel-1 bursts and orbits will be downloaded
DATADIR = 'data'
# path to the downloaded DEM file
DEM = f'{DATADIR}/dem.nc'
ZARRDIR = 'zarr'

## Download and Unpack Datasets

### Enter Your ASF User and Password

If the data directory is empty or doesn't exist, you'll need to download Sentinel-1 scenes from the Alaska Satellite Facility (ASF) datastore. Use your Earthdata Login credentials. If you don't have an Earthdata Login, you can create one at https://urs.earthdata.nasa.gov//users/new

You can also use pre-existing SLC scenes stored on your Google Drive, or you can copy them using a direct public link from iCloud Drive.

The credentials below are available at the time the notebook is validated.

In [ ]:
# Set these variables to None and you will be prompted to enter your username and password below.
asf_username = 'GoogleColab2023'
asf_password = 'GoogleColab_2023'

In [ ]:
# Set these variables to None and you will be prompted to enter your username and password below.
asf = ASF(asf_username, asf_password)
print(asf.download(DATADIR, BURSTS, n_jobs=2))

In [ ]:
# read the notices printed below carefully
s1 = S1(DATADIR)
s1.to_dataframe().to_file(f"{DATADIR}/s1.geojson", driver="GeoJSON")
#df = gpd.read_file("s1.geojson")
s1.to_dataframe()

In [ ]:
# scan the data directory for Sentinel-1 SLC bursts and download related orbits
EOF().download(DATADIR, s1.to_dataframe())

In [ ]:
s1.to_dataframe()

In [ ]:
# download Copernicus Global DEM 1 arc-second
Tiles().download_dem(s1.to_dataframe(), provider='GLO', filename=DEM).plot.imshow(cmap='cividis')

## Preprocess Sentinel-1 SLC and Store as Geocoded Zarr

In [ ]:
# scan SLC bursts with DEM
s1 = S1(DATADIR, DEM=DEM)
s1.to_dataframe()

In [ ]:
# preview selected bursts
s1.plot(ref='2019-07-02')

In [ ]:
# all in one command: load SLC bursts with DEM and transform to ZARR
s1.transform(ZARRDIR, ref='2019-07-02', remove_topo_phase=False)

# Stage 2. InSAR.dev: A Pure-Python package for InSAR processing

For InSAR analysis, only a pure-Python package is required—no binary installation needed—so processing can run on any Windows, macOS, or Linux host.

## Load and Setup Python Modules

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# setup dark theme
from insardev.UI import UI
UI('dark')

In [ ]:
import sys
!{sys.executable} -m pip install --no-cache-dir \
  "git+https://github.com/AlexeyPechnikov/InSARdev.git#subdirectory=insardev"
# print versions
from insardev import __version__ as insardev_version
from insardev_toolkit import __version__ as toolkit_version
print("insardev version:", insardev_version)
print("insardev_toolkit version:", toolkit_version)
# import modules to be used in the notebook
from insardev import Stack
# data downloader
from insardev_toolkit.HTTP import unzip

In [ ]:
# 3D plotting modules
import pyvista as pv
# white background
#pv.set_plot_theme("document")
pv.set_plot_theme("dark")
from IPython.display import display, HTML
if 'google.colab' in sys.modules:
    import panel
    panel.extension(comms='ipywidgets')
    panel.extension('vtk')

## Specify Directories

In [ ]:
ZARRDIR = 'zarr'

## Download and Unpack Datasets

We can process datasets hosted on GitHub, Zenodo, or other platforms in a single workflow.

In [ ]:
# example downloading command
#unzip("https://zenodo.org/records/15347694/files/Türkiye_Elevation-40x10-004.zip", ZARRDIR)

## Run Local Dask Cluster

Launch a Dask cluster for local or distributed multicore computing. This makes it possible to process terabyte-scale Sentinel-1 SLC datasets even on an Apple MacBook Air with 16 GB of RAM.

In [ ]:
# simple Dask initialization
if 'client' in globals():
    client.shutdown()
from dask.distributed import Client
client = Client(silence_logs='CRITICAL')
client

## InSAR.dev Processing

In [ ]:
stack = Stack()
stack.load(ZARRDIR)
stack

In [ ]:
stack.plot(cmap='autumn', alpha=0.15)

## Build Interferogram

In [ ]:
# take only intefrerograms as the first element [0], ignore correlation [1]
intf = stack.phasediff_multilook([1,0], wavelength=100, goldstein=32)[0].downsample(40).compute()
intf

In [ ]:
# unify bursts and downsample to 40m
intf = intf.align().dissolve().compute()

In [ ]:
intf.plot(alpha=0.8);

In [ ]:
stack.to_vtk('vtk', intf.downsample(100))

In [ ]:
# build interactive 3D plot
plotter = pv.Plotter(notebook=True)
plotter.add_mesh(pv.read('vtk/VV.vtk').scale([1, 1, 4], inplace=True), scalars='20190708_20190702', cmap='turbo', ambient=0.1, show_scalar_bar=True)
plotter.show_axes()
plot = plotter.show(screenshot='3D Interferogram.png', jupyter_backend='panel' if 'google.colab' in sys.modules else 'client', return_viewer=True)
if 'google.colab' in sys.modules:
    plot = panel.panel(
        plotter.render_window, orientation_widget=plotter.renderer.axes_enabled,
        enable_keybindings=False, sizing_mode='stretch_width', min_height=600
    )
display(HTML('<h1 style="text-align:center; margin-bottom:0;">Interactive Sentinel-1 Interferogram on DEM</h1>'))
plot

## Unwrap Interferogram to Phase

In [ ]:
phase = Stack(stack).unwrap2d(intf).compute()
phase

In [ ]:
# unify bursts
phase = phase.align().dissolve()

In [ ]:
phase.plot(vmin=-80, vmax=80, alpha=0.8);

In [ ]:
stack.to_vtk('phase', phase.downsample(100))

In [ ]:
# build interactive 3D plot
plotter = pv.Plotter(notebook=True)
plotter.add_mesh(pv.read('phase/VV.vtk').scale([1, 1, 4], inplace=True), scalars='20190708_20190702', cmap='turbo', ambient=0.1, show_scalar_bar=True)
plotter.show_axes()
plot = plotter.show(screenshot='3D Phase.png', jupyter_backend='panel' if 'google.colab' in sys.modules else 'client', return_viewer=True)
if 'google.colab' in sys.modules:
    plot = panel.panel(
        plotter.render_window, orientation_widget=plotter.renderer.axes_enabled,
        enable_keybindings=False, sizing_mode='stretch_width', min_height=600
    )
display(HTML('<h1 style="text-align:center; margin-bottom:0;">Interactive Sentinel-1 Phase on DEM</h1>'))
plot

## Convert Phase to Elevation

In [ ]:
# for reference, compute height for a single fringe
import numpy as np
baseline = intf.isel(0).data.BPR.round(3).item(0)
print ('baseline =', baseline)
height = stack.elevation(2*np.pi, baseline)
print('height for a single fringe =', height, 'meters')

In [ ]:
elevation = stack.elevation(phase)

In [ ]:
# select one burst for the original area
ele = elevation.isel(1).data.ele[0]
topo = stack.transform().isel(1).data.ele.interp_like(ele)

In [ ]:
points_topo = topo.values.ravel()
points_ele = ele.values.ravel()
nanmask = np.isnan(points_topo) | np.isnan(points_ele)
points_topo = points_topo[~nanmask]
points_ele = points_ele[~nanmask]

points_topo = points_topo - points_topo.mean()
points_ele = points_ele - points_ele.mean()

plt.figure(figsize=(12, 4), dpi=300)
plt.scatter(points_ele, points_topo, c='y', alpha=0.1, s=0.01)

# adding a 1:1 line
max_value = max(points_topo.max(), points_ele.max())
min_value = min(points_topo.min(), points_ele.min())
plt.plot([min_value, max_value], [min_value, max_value], 'w--')

plt.xlabel('Elevation, m', fontsize=16)
plt.ylabel('DEM, m', fontsize=16)
plt.title('Cross-Comparison between Sentinel-1 Elevation and Copernicus GLO-30 DEM', fontsize=18)
plt.grid(True)
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 6))

topo_y = (topo - topo.mean()).sel(y=4400000, method='nearest')
ele_y = (ele - ele.mean()).sel(y=4400000, method='nearest')
topo_y.plot(ax=axes[0], color='w', ls='-', lw=1, label='Copernicus GLO-30 DEM')
ele_y.plot(ax=axes[0], color='y', ls='-', lw=1, label='Sentinel-1 Elevation')
axes[0].set_title('Cross-Section at y=4400000')
axes[0].legend()

topo_x = (topo - topo.mean()).sel(x=570000, method='nearest')
ele_x = (ele - ele.mean()).sel(x=570000, method='nearest')
topo_x.plot(ax=axes[1], color='w', ls='-', lw=1, label='Copernicus GLO-30 DEM')
ele_x.plot(ax=axes[1], color='y', ls='-', lw=1, label='Sentinel-1 Elevation')
axes[1].set_title('Cross-Section at x=570000')
axes[1].legend()

plt.suptitle('Comparison of Sentinel-1 Elevation and Copernicus GLO-30 DEM')
plt.tight_layout()
plt.show()

In [ ]:
stack.to_vtk('elevation', elevation.downsample(100))
stack.to_vtk('topo', stack.transform()[['ele']].downsample(100).align().dissolve())

In [ ]:
# build interactive 3D plot
plotter = pv.Plotter(shape=(1, 2), notebook=True)
plotter.subplot(0, 0)
plotter.add_mesh(pv.read('elevation/ele.vtk').scale([1, 1, 4], inplace=True), scalars='20190708_20190702', cmap='turbo', ambient=0.1, show_scalar_bar=True)
plotter.subplot(0, 1)
plotter.add_mesh(pv.read('topo/ele.vtk').scale([1, 1, 4], inplace=True), scalars='ele', cmap='turbo', ambient=0.1, show_scalar_bar=True)
plotter.show_axes()
plot = plotter.show(screenshot='3D Elevation.png', jupyter_backend='panel' if 'google.colab' in sys.modules else 'client', return_viewer=True)
if 'google.colab' in sys.modules:
    plot = panel.panel(
        plotter.render_window, orientation_widget=plotter.renderer.axes_enabled,
        enable_keybindings=False, sizing_mode='stretch_width', min_height=600
    )
display(HTML('<h1 style="text-align:center; margin-bottom:0;">Interactive Sentinel-1 Elevation vs GLO-30 DEM</h1>'))
plot

## Save the Results

Save the results in geospatial data formats like to NetCDF, GeoTIFF and others. The both formats (NetCDF and GeoTIFF) can be opened in QGIS and other GIS applications.

In [ ]:
elevation.to_dataset().ele[0].rio.to_raster('elevation.tif')
#elevation.align().to_dataset().ele[0].to_netcdf('elevation.nc')

## Export from Google Colab

In [ ]:
if 'google.colab' in sys.modules:
    from google.colab import files
    files.download('elevation/ele.vtk')
    #files.download('elevation.nc')